In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import f1_score, classification_report

from sklearn import svm
import gensim.models as g
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import auc, roc_curve, roc_auc_score

from nltk.corpus import wordnet

In [2]:
data = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/preprocessed_cleaned_body.csv')

In [3]:
wordsToCut = set(stopwords.words('english'))

def getPOS(words):
    #return nltk.pos_tag(word)
    return [(word, get_wordnet_pos(word)) for word in words]

def getIndividualPOS(word):
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV} 
    tag = nltk.pos_tag(word[0].upper())
    return tag_dict.get(tag)

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    return tag_dict.get(tag, wordnet.NOUN)

for index, row in data.iterrows():
    
    #removing stopwords and punctuation
    tokenized = nltk.word_tokenize(row['cleaned_body'])
    temp = [word for word in tokenized if word.lower() not in wordsToCut and word not in string.punctuation and word.isalpha()]
    
    #row['Body'] = ' '.join(temp)
    
    #row['Body'] = [word for word in nltk.word_tokenize(row['Body']) if word not in wordsToCut]
    #row['Body'] = row['Body'].lower()
    #nopunc = [char for char in row['Body'] if char not in string.punctuation]
    #nopunc.join('')
    
    #lemmatizing, ONLY KEEPING ENTITIES
    lemmatizer = WordNetLemmatizer()
    partsOfSpeech = getPOS(temp)
    
    #lemmatized = [lemmatizer.lemmatize(word, pos) for word, pos in partsOfSpeech if pos == 'n']
    lemmatized = [lemmatizer.lemmatize(word, pos) for word, pos in partsOfSpeech]
    final = ' '.join(lemmatized)
    data.at[index, 'cleaned_body'] = final
    

In [4]:
right = data.loc[data['Bias'] >= 5]
neutral = data.loc[(data['Bias'] < 5) & (data['Bias'] > -5)]
left = data.loc[data['Bias'] <= -5]
print (len(right) + len(neutral) + len(left))

1675


In [5]:
right['Bias'] = 1
neutral['Bias'] = 0
left['Bias'] = -1

/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [6]:
cat_df = pd.concat([right, neutral, left])
cat_df

,Body,Bias,cleaned_body
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
155,Asked by if he believes President Trump's emer...,1,ask belief president trump emergency declarati...
156,You May Like\nThese 10 Shawshank Redemption Fa...,1,may like shawshank redemption fact insane hive...
157,"NEW: This Germantown, Md. man tells he was att...",1,new germantown md man tell wa attack saturday ...
158,“A little as two to three milligrams of fentan...,1,little two three milligram fentanyl induce res...
...,...,...,...
1655,"In any sane reality, last week's vote to hold ...",-1,sane reality last week vote hold new election ...
1656,People have expressed concern that Lindsey Gra...,-1,people express concern lindsey graham shameles...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,-1,day big coverup nigh thursday anticipate relea...
1658,"If you are a man or a lady of letters, and are...",-1,man lady letter also verse internet tradition ...


In [7]:
vectorizer = TfidfVectorizer(max_features=800)
tfidf = vectorizer.fit_transform(cat_df['cleaned_body'])
X = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names())
y = cat_df['Bias']

In [8]:
X

,ability,able,abortion,abuse,access,accord,account,accuse,across,act,...,world,would,write,wrong,year,yearold,yet,york,young,zealand
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.031533,...,0.030346,0.000000,0.000000,0.000000,0.058644,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.080040,0.059389,0.000000,0.027418,0.0,0.0,0.000000,0.0,0.0
1671,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.023663,0.000000,0.111007,0.024318,0.0,0.0,0.038913,0.0,0.0
1672,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.073709,0.036461,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
1673,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.117083,0.000000,0.091540,0.000000,0.0,0.0,0.000000,0.0,0.0


In [9]:
col_names = X.columns

scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)
X = pd.DataFrame(scaled, columns=col_names)

In [10]:
X

,ability,able,abortion,abuse,access,accord,account,accuse,across,act,...,world,would,write,wrong,year,yearold,yet,york,young,zealand
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.137554,...,0.081576,0.000000,0.000000,0.000000,0.134333,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.259053,0.154897,0.000000,0.062805,0.0,0.0,0.000000,0.0,0.0
1671,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.076587,0.000000,0.565626,0.055704,0.0,0.0,0.121456,0.0,0.0
1672,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.238562,0.095097,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
1673,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.378942,0.000000,0.466437,0.000000,0.0,0.0,0.000000,0.0,0.0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [12]:
model = svm.SVC(probability = True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))

Accuracy score:  0.6085918854415274


In [13]:
y_probs = model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_probs, average="macro", multi_class="ovo")
print('ROC Score is: ', roc_auc)

ROC Score is:  0.7839574640056961


In [14]:
right

,Body,Bias,cleaned_body
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
155,Asked by if he believes President Trump's emer...,1,ask belief president trump emergency declarati...
156,You May Like\nThese 10 Shawshank Redemption Fa...,1,may like shawshank redemption fact insane hive...
157,"NEW: This Germantown, Md. man tells he was att...",1,new germantown md man tell wa attack saturday ...
158,“A little as two to three milligrams of fentan...,1,little two three milligram fentanyl induce res...
...,...,...,...
1670,Only hours after the first flames that eventua...,1,hour first flame eventually would consume notr...
1671,Washington watchdog Judicial Watch believes th...,1,washington watchdog judicial watch belief fbi ...
1672,"Two Fox News personalities, as Notre Dame Cath...",1,two fox news personality notre dame cathedral ...
1673,Chase Bank is shutting down accounts of people...,1,chase bank shut account people organization co...


In [15]:
neutral

,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,0,abortion right advocate ask u supreme court su...
1,A federal appeals court rejected the most dire...,0,federal appeal court reject direct constitutio...
2,As part of the Trump administration's effort t...,0,part trump administration effort slow migrant ...
3,"President Donald Trump and ""the Trump of the T...",0,president donald trump trump tropic brazilian ...
5,Facebook announced Tuesday that it will block ...,0,facebook announce tuesday block landlord credi...
...,...,...,...
1633,"In that previous case, the town of Greece, in ...",0,previous case town greece upstate new york ope...
1636,This combination of undated photos released by...,0,combination undated photo release jefferson co...
1641,The prototypes built as part of President Trum...,0,prototype built part president trump border wa...
1661,"(PASTEMAGAZINE) — Monday night, Bernie Sanders...",0,pastemagazine monday night bernie sander town ...


In [16]:
left

,Body,Bias,cleaned_body
4,"U.S Senator Elizabeth Warren, who is competing...",-1,u senator elizabeth warren compete democratic ...
10,Presidential candidates submitted their first ...,-1,presidential candidate submit first quarter fu...
11,Even as President Donald Trump continues his f...,-1,even president donald trump continue feud rep ...
12,President Donald Trump again criticized the la...,-1,president donald trump criticize late sen john...
14,U.S. cyber operators disrupted internet access...,-1,u cyber operator disrupt internet access purpo...
...,...,...,...
1655,"In any sane reality, last week's vote to hold ...",-1,sane reality last week vote hold new election ...
1656,People have expressed concern that Lindsey Gra...,-1,people express concern lindsey graham shameles...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,-1,day big coverup nigh thursday anticipate relea...
1658,"If you are a man or a lady of letters, and are...",-1,man lady letter also verse internet tradition ...


In [17]:
type(X_train)

pandas.core.frame.DataFrame

### train with kfold cross validation

In [18]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2)
